In [2]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import *

In [3]:
seed = 7

In [4]:
import warnings
warnings.filterwarnings('ignore')

# Collect data

## Define path

In [6]:
root_data='comodity-price-prediction-penyisihan-arkavidia-9'
hrg_bhn_pangan='Harga Bahan Pangan'
folder_train='train'
folder_mata_uang='Mata Uang'
folder_google_trend='Google Trend'
folder_test='test'

## Define date range

In [7]:
tanggal_awal=dt.datetime(year=2024,month=10,day=1)
tanggal_akhir=dt.datetime(year=2024,month=12,day=31)

# Define list of basic needs

In [8]:
list_bhn_pgn=[
    'Bawang Merah',         # bawang merah
    'Bawang Putih Bonggol', # bawang putih
    'Beras Medium',         # beras
    'Beras Premium',        # beras
    'Cabai Merah Keriting', # cabai merah
    'Cabai Rawit Merah',    # cabai rawit
    'Daging Ayam Ras',      # daging ayam
    'Daging Sapi Murni',    # daging sapi
    'Gula Konsumsi',        # gula
    'Minyak Goreng Curah',  # minyak goreng
    'Minyak Goreng Kemasan Sederhana',  #minyak goreng
    'Telur Ayam Ras',       # telur ayam
    'Tepung Terigu (Curah)',# tepung terigu
]

In [9]:
list_google_trend={
    'Bawang Merah': 'bawang merah',
    'Bawang Putih Bonggol': 'bawang putih',
    'Beras Medium':'beras',
    'Beras Premium':'beras',
    'Cabai Merah Keriting':'cabai merah',
    'Cabai Rawit Merah':'cabai rawit',
    'Daging Ayam Ras':'daging ayam',
    'Daging Sapi Murni':'daging sapi',
    'Gula Konsumsi':'gula',
    'Minyak Goreng Curah':'minyak goreng',
    'Minyak Goreng Kemasan Sederhana':'minyak goreng',
    'Telur Ayam Ras':'telur ayam',
    'Tepung Terigu (Curah)':'tepung terigu',
}

In [10]:
list_nama_daerah=[
    'Aceh',
    'Bali',
    'Banten',
    'Bengkulu',
    'DI Yogyakarta',
    'DKI Jakarta',
    'Gorontalo',
    'Jambi',
    'Jawa Barat',
    'Jawa Tengah',
    'Jawa Timur',
    'Kalimantan Barat',
    'Kalimantan Selatan',
    'Kalimantan Tengah',
    'Kalimantan Timur',
    'Kalimantan Utara',
    'Kepulauan Bangka Belitung',
    'Kepulauan Riau',
    'Lampung',
    'Maluku Utara',
    'Maluku',
    'Nusa Tenggara Barat',
    'Nusa Tenggara Timur',
    'Papua Barat',
    'Papua',
    'Riau',
    'Sulawesi Barat',
    'Sulawesi Selatan',
    'Sulawesi Tengah',
    'Sulawesi Tenggara',
    'Sulawesi Utara',
    'Sumatera Barat',
    'Sumatera Selatan',
    'Sumatera Utara'
]

In [13]:
path_data_source=os.path.join(root_data,hrg_bhn_pangan,folder_test)

In [40]:
data_bhn_pgn={
    'Tanggal':[],
    'Bahan_pangan':[],
    'Daerah':[],
    'Harga_lag_1':[],
    'Harga_lag_7':[],
    'Harga_lag_30':[],
    'Google_trends':[],
    'MYRUSD':[],
    'SGDUSD':[],
    'THBUSD':[],
    'USDIDR':[],
    'Harga':[]
}
df_bhn_pgn_all=pd.DataFrame(data_bhn_pgn)

In [28]:
myrusd_path=os.path.join(root_data,folder_mata_uang,'MYRUSD=X.csv')
df_myrusd=pd.read_csv(myrusd_path)
df_myrusd['Date']=pd.to_datetime(df_myrusd['Date'])
sgdusd_path=os.path.join(root_data,folder_mata_uang,'SGDUSD=X.csv')
df_sgdusd=pd.read_csv(sgdusd_path)
df_sgdusd['Date']=pd.to_datetime(df_sgdusd['Date'])
thbusd_path=os.path.join(root_data,folder_mata_uang,'THBUSD=X.csv')
df_thbusd=pd.read_csv(thbusd_path)
df_thbusd['Date']=pd.to_datetime(df_thbusd['Date'])
usdidr_path=os.path.join(root_data,folder_mata_uang,'USDIDR=X.csv')
df_usdidr=pd.read_csv(usdidr_path)
df_usdidr['Date']=pd.to_datetime(df_usdidr['Date'])

myrusd=3349.6199
sgdusd=11687.3767
thbusd=449.8396
usdidr=15425.0

# Process Collect Data From All Files

In [31]:
for date in [tanggal_awal + dt.timedelta(days=x) for x in range((tanggal_akhir - tanggal_awal).days + 1)]:
    for nama_bhn_pgn in list_bhn_pgn:
        file_bhn_pgn=os.path.join(path_data_source, nama_bhn_pgn+'.csv')
        df_bhn_pgn=pd.read_csv(file_bhn_pgn)
        try:
            df_bhn_pgn['Date']=pd.to_datetime(df_bhn_pgn['Date'])
        except Exception as e:
            df_bhn_pgn['Tanggal']=pd.to_datetime(df_bhn_pgn['Tanggal'])
        yesterday = date - timedelta(days=1)
        week_ago = date - timedelta(days=7)
        month_ago = date - relativedelta(months=1)
        for daerah in list_nama_daerah:
            try:
                gt_path=os.path.join(root_data,folder_google_trend,list_google_trend[nama_bhn_pgn],daerah+'.csv')
                df_gt=pd.read_csv(gt_path)
                df_gt['Date']=pd.to_datetime(df_gt['Date'])
            except Exception as e:
                df_gt=None
            
            # QUERY DATA KURS
            if not df_myrusd.query('Date == @date').empty and not df_usdidr.query('Date == @date').empty:
                myrusd=round(df_myrusd.query('Date == @date')['Adj Close'].values[0]*df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_myrusd.query('Date == @date').empty else np.int32(0)
            if not df_sgdusd.query('Date == @date').empty and not df_usdidr.query('Date == @date').empty:
                sgdusd=round(df_sgdusd.query('Date == @date')['Adj Close'].values[0]*df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_sgdusd.query('Date == @date').empty else np.int32(0)
            if not df_thbusd.query('Date == @date').empty and not df_usdidr.query('Date == @date').empty:
                thbusd=round(df_thbusd.query('Date == @date')['Adj Close'].values[0]*df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_thbusd.query('Date == @date').empty else np.int32(0)
            if not df_usdidr.query('Date == @date').empty:
                usdidr=round(df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_myrusd.query('Date == @date').empty else np.int32(0)
            
            new_data={
                'Tanggal':date,
                'Bahan_pangan':nama_bhn_pgn,
                'Daerah':daerah,
                'Harga_lag_1':round(df_bhn_pgn.query('Date == @yesterday')[daerah].fillna(0).values[0]) if not df_bhn_pgn.query('Date == @yesterday').empty else 0,
                'Harga_lag_7':round(df_bhn_pgn.query('Date == @week_ago')[daerah].fillna(0).values[0]) if not df_bhn_pgn.query('Date == @week_ago').empty else 0,
                'Harga_lag_30':round(df_bhn_pgn.query('Date == @month_ago')[daerah].fillna(0).values[0]) if not df_bhn_pgn.query('Date == @month_ago').empty else 0,
                'Google_trends': round(df_gt.query('Date == @date')[list_google_trend[nama_bhn_pgn]].fillna(0).values[0]) if df_gt is not None and not df_gt.query('Date == @date').empty else 0,
                'MYRUSD':myrusd,
                'SGDUSD':sgdusd,
                'THBUSD':thbusd,
                'USDIDR':usdidr,
                'Harga':round(df_bhn_pgn.query('Date == @date')[daerah].fillna(0).values[0]) if not df_bhn_pgn.query('Date == @date').empty else 0
            }
            df_bhn_pgn_all = df_bhn_pgn_all._append(new_data, ignore_index=True)

## Retrieve data for forecasting

In [18]:
# Check columns in csv, are these columns is match with data from array
for nama_bhn_pgn in list_bhn_pgn:
    file_bhn_pgn=os.path.join(path_data_source, nama_bhn_pgn+'.csv')
    df_bhn_pgn=pd.read_csv(file_bhn_pgn)
    df_bhn_pgn=df_bhn_pgn.drop('Date', axis=1)
    print(f'Length of data {nama_bhn_pgn}: {np.array_equal(list_nama_daerah, df_bhn_pgn.columns)}\n')

Length of data Bawang Merah: True

Length of data Bawang Putih Bonggol: True

Length of data Beras Medium: True

Length of data Beras Premium: True

Length of data Cabai Merah Keriting: True

Length of data Cabai Rawit Merah: True

Length of data Daging Ayam Ras: True

Length of data Daging Sapi Murni: True

Length of data Gula Konsumsi: True

Length of data Minyak Goreng Curah: True

Length of data Minyak Goreng Kemasan Sederhana: True

Length of data Telur Ayam Ras: True

Length of data Tepung Terigu (Curah): True



## Load data training

In [31]:
data_training=pd.read_csv('train.csv')
data_training['Tanggal'] = pd.to_datetime(data_training['Tanggal'], format='%Y-%m-%d', errors='coerce')

## Process for data forecasting

In [41]:
new_data_list=[]
for index, date in enumerate([tanggal_awal + dt.timedelta(days=x) for x in range((tanggal_akhir - tanggal_awal).days + 1)]):
    if index%100:
        print(new_data_list[index])
    for nama_bhn_pgn in list_bhn_pgn:
        yesterday = date - timedelta(days=1)
        week_ago = date - timedelta(days=7)
        month_ago = date - relativedelta(months=1)
        for daerah in list_nama_daerah:
            new_data={
                'Tanggal':date,
                'Bahan_pangan':nama_bhn_pgn,
                'Daerah':daerah,
                'Harga_lag_1':round(data_training.query('Tanggal == @yesterday and Daerah == @daerah and Bahan_pangan == @nama_bhn_pgn')['Harga'].values[0]) if not data_training.query('Tanggal == @yesterday').empty else 0,
                'Harga_lag_7':round(data_training.query('Tanggal == @week_ago and Daerah == @daerah and Bahan_pangan == @nama_bhn_pgn')['Harga'].values[0]) if not data_training.query('Tanggal == @week_ago').empty else 0,
                'Harga_lag_30':round(data_training.query('Tanggal == @month_ago and Daerah == @daerah and Bahan_pangan == @nama_bhn_pgn')['Harga'].values[0]) if not data_training.query('Tanggal == @month_ago').empty else 0,
                'Google_trends': 0,
                'Harga': 0
            }
            new_data_list.append(new_data)

{'Tanggal': datetime.datetime(2024, 10, 1, 0, 0), 'Bahan_pangan': 'Bawang Merah', 'Daerah': 'Bali', 'Harga_lag_1': 24230, 'Harga_lag_7': 24240, 'Harga_lag_30': 21250, 'Google_trends': 0, 'Harga': 0}
{'Tanggal': datetime.datetime(2024, 10, 1, 0, 0), 'Bahan_pangan': 'Bawang Merah', 'Daerah': 'Banten', 'Harga_lag_1': 27940, 'Harga_lag_7': 25910, 'Harga_lag_30': 25520, 'Google_trends': 0, 'Harga': 0}
{'Tanggal': datetime.datetime(2024, 10, 1, 0, 0), 'Bahan_pangan': 'Bawang Merah', 'Daerah': 'Bengkulu', 'Harga_lag_1': 27390, 'Harga_lag_7': 26960, 'Harga_lag_30': 23840, 'Google_trends': 0, 'Harga': 0}
{'Tanggal': datetime.datetime(2024, 10, 1, 0, 0), 'Bahan_pangan': 'Bawang Merah', 'Daerah': 'DI Yogyakarta', 'Harga_lag_1': 24450, 'Harga_lag_7': 22240, 'Harga_lag_30': 19600, 'Google_trends': 0, 'Harga': 0}
{'Tanggal': datetime.datetime(2024, 10, 1, 0, 0), 'Bahan_pangan': 'Bawang Merah', 'Daerah': 'DKI Jakarta', 'Harga_lag_1': 30370, 'Harga_lag_7': 29880, 'Harga_lag_30': 29400, 'Google_trends'

In [42]:
# date=dt.datetime(year=2024,month=10,day=1)
# week_ago = date - timedelta(days=7)
# daerah=list_nama_daerah[0]
# bhn_pgn=list_bhn_pgn[0]
# data=round(data_training.query('Tanggal == @week_ago and Daerah == @daerah and Bahan_pangan == @bhn_pgn')['Harga'].values[0]) if not data_training.query('Tanggal == @week_ago').empty else 0
# print(data)
df_bhn_pgn_all = pd.DataFrame(new_data_list)

In [45]:
df_bhn_pgn_all.to_csv('data_forecast1.csv', index=False)

In [29]:
data_training.tail()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,MYRUSD,SGDUSD,THBUSD,USDIDR,Harga
471044,2024-09-30,Tepung Terigu (Curah),Sulawesi Tenggara,9680.0,9720.0,9810.0,0.0,3670.3084,11804.482,467.3261,15118.0,9620.0
471045,2024-09-30,Tepung Terigu (Curah),Sulawesi Utara,10760.0,10760.0,10770.0,0.0,3670.3084,11804.482,467.3261,15118.0,10690.0
471046,2024-09-30,Tepung Terigu (Curah),Sumatera Barat,9830.0,9760.0,9800.0,0.0,3670.3084,11804.482,467.3261,15118.0,9920.0
471047,2024-09-30,Tepung Terigu (Curah),Sumatera Selatan,8960.0,8940.0,8970.0,0.0,3670.3084,11804.482,467.3261,15118.0,8960.0
471048,2024-09-30,Tepung Terigu (Curah),Sumatera Utara,10710.0,10520.0,10870.0,0.0,3670.3084,11804.482,467.3261,15118.0,10640.0


In [19]:
pd.read_csv('train.csv').columns

Index(['Tanggal', 'Bahan_pangan', 'Daerah', 'Harga_lag_1', 'Harga_lag_7',
       'Harga_lag_30', 'Google_trends', 'MYRUSD', 'SGDUSD', 'THBUSD', 'USDIDR',
       'Harga'],
      dtype='object')

## Check Data Retrieved

In [32]:
df_bhn_pgn_all.head()

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,MYRUSD,SGDUSD,THBUSD,USDIDR,Harga
0,2024-01-01,Bawang Merah,Aceh,36790.0,34590.0,31080.0,0.0,3349.6133,11662.3455,446.9233,15383.0996,37020.0
1,2024-01-01,Bawang Merah,Bali,35340.0,29420.0,23510.0,0.0,3349.6133,11662.3455,446.9233,15383.0996,35020.0
2,2024-01-01,Bawang Merah,Banten,34060.0,32660.0,29300.0,0.0,3349.6133,11662.3455,446.9233,15383.0996,34540.0
3,2024-01-01,Bawang Merah,Bengkulu,36570.0,34540.0,29370.0,0.0,3349.6133,11662.3455,446.9233,15383.0996,36570.0
4,2024-01-01,Bawang Merah,DI Yogyakarta,33890.0,33420.0,25360.0,0.0,3349.6133,11662.3455,446.9233,15383.0996,34630.0


## Append data to csv

In [34]:
df_bhn_pgn_all.to_csv('tes.csv', mode='a', index=False, header=False)

In [16]:
file_bhn_pgn=os.path.join(path_data_source, list_bhn_pgn[0]+'.csv')
df_bhn_pgn=pd.read_csv(file_bhn_pgn)
gt_path=os.path.join(root_data,folder_google_trend,list_google_trend[list_bhn_pgn[0]],list_nama_daerah[0]+'.csv')
print(f'gt_path: {gt_path}')
df_gt=pd.read_csv(gt_path)
df_gt['Date']=pd.to_datetime(df_gt['Date'])
date=dt.datetime(year=2022,month=7,day=26)
df_gt.query('Date == @date')['bawang merah'].values[0]

gt_path: comodity-price-prediction-penyisihan-arkavidia-9\Google Trend\bawang merah\Aceh.csv


np.float64(49.50915254237288)